## Guide PyInstaller - Comment déployer `.py` en `.exe`

⚠️ *Pour que ce notebook soit fonctionnel, copiez le chemin absolu où il se trouve sur votre machine et remplacez la valeur de la variable `notebook_path` ci-dessous.*

In [3]:
#notebook_path = "REPLACE ME !"
notebook_path = "C:/Users/boyan/Projects/Python_Projects/Thales_Automation/pyinstaller/PyInstaller_Guide.ipynb" # Path to the pyinstaller folder on your computer

*Ensuite, ré-exécutez le notebook ("run all cells") et vous pourrez poursuivre votre lecture.*

### Pourquoi PyInstaller ?

Ce dossier rassemble des scripts d'installation et un guide d'usage pour **PyInstaller**, une lib python visant à déployer des scripts `.py` en `.exe`. <br>
L'extension `.exe` est standalone, c'est-à-dire que l'environnement utilisé n'a pas besoin de dépendences pour pouvoir exécuter le script.



**Attention :** PyInstaller gènère des .exe pour l'OS de la machine sur laquelle le script est compilé. <br> 
Un `.exe` généré sous un OS Winx64 ne pourra être exécuté que sur une autre machine Windows 64 bits. <br> 
Une bonne pratique est donc d'inclure dans le nom du fichier le **nom de l'OS** et la **version de Python** utilisée pour le générer.

Lire la doc officielle : [RTFM](https://pyinstaller.org)

### Installation Guidée

#### Setup

Imports

In [16]:
import os
import sys 
import subprocess

On vérifie l'arborescence des dossiers autour du notebook et on définit le  `path` , qui pointe vers  `pyinstaller` , afin que les chemins relatifs utilisés après soient corrects.

In [5]:
#TODO Check for worktree
if notebook_path == "REPLACE ME !":
    print("Pour que ce notebook soit fonctionnel, copiez le chemin absolu où il se trouve sur votre machine et remplacez la valeur de la variable path en début de notebook.")    
else:
    path = os.path.dirname(notebook_path)
    os.chdir(path)
    print(os.getcwd())
    if os.path.basename(os.path.normpath(path)) == 'pyinstaller':
        print('Folder is initialized correctly.')
    else: 
        print('Error: file was placed in the wrong dir.')

C:\Users\boyan\Projects\Python_Projects\Thales_Automation\pyinstaller
Folder is initialized correctly.


#### Version de Python

On vérifie tout d'abord la version de Python utilisée. <br>

In [6]:
python_version = subprocess.getoutput("python --version")
print(python_version)

Python 3.10.11


#### Versions disponibles de PyInstaller et ses dépendences

Le dossier `pyinstaller` contient une bibliothèque de versions de PyInstaller et de ses dépendences. <br>
On récupère ici la liste des versions disponibles.

In [7]:
libs_path = os.path.join(path,'libs')
os.chdir(libs_path)
available_versions = os.listdir()
print(available_versions)

['Python 3.10.11', 'Python 3.11', 'Python 3.6', 'Python 3.7', 'Python 3.8', 'Python 3.9', 'Python 3.9.7']


#### Compatibilité des versions disponibles

##### Fonctions

In [8]:
def createRequirementsFile(version_path):
    filename = os.path.join(version_path, "requirements.txt")
    with open(filename, "w") as f:
        f.write(f"pyinstaller \n")

In [9]:
#TODO: If problems of incompatible install arise, try using "py -x.x.x download", so that the interpreter is the right version for the libs
# pip download -r "C:\Users\boyan\Projects\Python_Projects\Thales_Automation\pyinstaller\libs\Python 3.7\requirements.txt" -d "C:\Users\boyan\Projects\Python_Projects\Thales_Automation\pyinstaller\libs\Python 3.7" --python-version 3.7  --only-binary=:all:

def downloadLibs(version_path,python_version):
    version_num = python_version.split(' ')[1]
    req_path = os.path.join(version_path, "requirements.txt")
    try:
        subprocess.check_call([sys.executable,'-m','pip','download','-r',req_path,'-d',version_path,'--python-version',version_num,'--only-binary=:all:'])
        return True
    except:
        return False

In [10]:
#TODO : Add a check to see if the version is already downloaded
#TODO : Add a check if the folder is already created
#TODO : Add a check if the folder is empty
#TODO : Add a check if the requirements.txt is already created
#TODO : Add a check if some files are missing

def downloadVersion(libs_path,python_version):
    print(f"Création du dossier pour {python_version}")
    version_path = os.path.join(libs_path, python_version)
    os.mkdir(version_path)
    os.chdir(version_path)
    print("Création du requirements.txt")
    createRequirementsFile(version_path)
    print("Téléchargement de PyInstaller et ses dépendences...")
    return downloadLibs(version_path, python_version)

In [11]:
def installVersion(version_path,python_version):
    version_path = os.path.join(version_path,python_version)
    os.chdir(version_path)
    try:
        subprocess.run(["pip", "install", "-r", "requirements.txt", "--target", version_path])
        return True
    except:
        return False

In [12]:
#DEBUG
print(libs_path)
print(python_version)
db_dir = 'C:/Users/boyan/Projects/Python_Projects/Thales_Automation/pyinstaller/libs'  
#createRequirementsFile(db_dir)
#downloadLibs(db_dir,'Python 3.7')
#downloadVersion(libs_path,'Python 3.7')

C:/Users/boyan/Projects/Python_Projects/Thales_Automation/pyinstaller\libs
Python 3.10.11


##### Script d'Installation

Ce script assure l'installation de PyInstaller si une version compatible à la version de Python utilisée est disponible. <br>

In [13]:
'''
python_global_version = 'Python '+ '.'.join(python_version.split()[1].split('.')[0:2])
available_global_versions = ['Python '+ '.'.join(version.split()[1].split('.')[0:2]) for version in available_versions]

if python_version in available_versions:
    print(f"PyInstaller est disponible localement pour {python_version}.")
    print("Installation de PyInstaller...")
    
    if installVersion(libs_path,python_version):
        print("PyInstaller installé avec succès.")
        
    else:
        print("Erreur lors de l'installation de PyInstaller.")
    

if python_global_version in available_global_versions:
    print(f"PyInstaller n'est pas disponible localement pour {python_version}, mais il y a une version pour {python_global_version}.")
    print("Installation de PyInstaller...")
    
    if installVersion(libs_path,python_global_version):
        print("PyInstaller installé avec succès.")
        
    else:
        print("Erreur lors de l'installation de PyInstaller.")
    
else:
    print(f"PyInstaller n'est pas disponible localement pour {python_version} ou {python_global_version}.")
    print(f"Tentative de téléchargement de PyInstaller pour {python_version} depuis PyPi.org ...")
    
    if downloadVersion(libs_path,python_version):
        print(f"Téléchargement de PyInstaller pour {python_version} réussi.")
        print(f"Relancez le script pour l'installer.")
        
    else:
        print("Téléchargement impossible pour cause d'erreur réseau.")
        print("Le dossier de version a été crée, et un 'requirements.txt' y a été ajouté.")
        print("Veuillez relancer le script sur une machine connectée à internet.")
        print("Les librairies seront automatiquement téléchargées dans le dossier de version.")
        print(f"Alternativement, les librairies peuvent être placées manuellement dans le dossier {os.path.join(libs_path,python_version)}")  
'''    
        

'\npython_global_version = \'Python \'+ \'.\'.join(python_version.split()[1].split(\'.\')[0:2])\navailable_global_versions = [\'Python \'+ \'.\'.join(version.split()[1].split(\'.\')[0:2]) for version in available_versions]\n\nif python_version in available_versions:\n    print(f"PyInstaller est disponible localement pour {python_version}.")\n    print("Installation de PyInstaller...")\n    \n    if installVersion(libs_path,python_version):\n        print("PyInstaller installé avec succès.")\n        \n    else:\n        print("Erreur lors de l\'installation de PyInstaller.")\n    \n\nif python_global_version in available_global_versions:\n    print(f"PyInstaller n\'est pas disponible localement pour {python_version}, mais il y a une version pour {python_global_version}.")\n    print("Installation de PyInstaller...")\n    \n    if installVersion(libs_path,python_global_version):\n        print("PyInstaller installé avec succès.")\n        \n    else:\n        print("Erreur lors de l\'in

##### Script de Téléchargement

Ce script permet de télécharger les versions de PyInstaller et de ses dépendences qui ne sont pas déjà disponibles dans le dossier `pyinstaller`. <br>

In [14]:
'''
python_global_version = 'Python '+ '.'.join(python_version.split()[1].split('.')[0:2])
available_global_versions = ['Python '+ '.'.join(version.split()[1].split('.')[0:2]) for version in available_versions]

if python_version in available_versions:
    print(f"PyInstaller est disponible localement pour {python_version}.")
    print("Installation de PyInstaller...")
    
    if installVersion(libs_path,python_version):
        print("PyInstaller installé avec succès.")
        
    else:
        print("Erreur lors de l'installation de PyInstaller.")
    

if python_global_version in available_global_versions:
    print(f"PyInstaller n'est pas disponible localement pour {python_version}, mais il y a une version pour {python_global_version}.")
    print("Installation de PyInstaller...")
    
    if installVersion(libs_path,python_global_version):
        print("PyInstaller installé avec succès.")
        
    else:
        print("Erreur lors de l'installation de PyInstaller.")
    
else:
    print(f"PyInstaller n'est pas disponible localement pour {python_version} ou {python_global_version}.")
    print(f"Tentative de téléchargement de PyInstaller pour {python_version} depuis PyPi.org ...")
    
    if downloadVersion(libs_path,python_version):
        print(f"Téléchargement de PyInstaller pour {python_version} réussi.")
        print(f"Relancez le script pour l'installer.")
        
    else:
        print("Téléchargement impossible pour cause d'erreur réseau.")
        print("Le dossier de version a été crée, et un 'requirements.txt' y a été ajouté.")
        print("Veuillez relancer le script sur une machine connectée à internet.")
        print("Les librairies seront automatiquement téléchargées dans le dossier de version.")
        print(f"Alternativement, les librairies peuvent être placées manuellement dans le dossier {os.path.join(libs_path,python_version)}")  
    
'''       

'\npython_global_version = \'Python \'+ \'.\'.join(python_version.split()[1].split(\'.\')[0:2])\navailable_global_versions = [\'Python \'+ \'.\'.join(version.split()[1].split(\'.\')[0:2]) for version in available_versions]\n\nif python_version in available_versions:\n    print(f"PyInstaller est disponible localement pour {python_version}.")\n    print("Installation de PyInstaller...")\n    \n    if installVersion(libs_path,python_version):\n        print("PyInstaller installé avec succès.")\n        \n    else:\n        print("Erreur lors de l\'installation de PyInstaller.")\n    \n\nif python_global_version in available_global_versions:\n    print(f"PyInstaller n\'est pas disponible localement pour {python_version}, mais il y a une version pour {python_global_version}.")\n    print("Installation de PyInstaller...")\n    \n    if installVersion(libs_path,python_global_version):\n        print("PyInstaller installé avec succès.")\n        \n    else:\n        print("Erreur lors de l\'in

##### Script de Téléchargement Auto

Ce script passe en revue les versions de PyInstaller précédemment installées et télécharge les fichiers ou versions manquantes. <br>

In [31]:
number_of_files = -1
os.chdir(libs_path)
for version in available_versions:
    number_of_files = max(number_of_files,len(os.listdir("./"+version)))
    # folder is empty
    if os.listdir("./"+version) == []:
        print(f'{version} empty')
        break
    # folder contains a wrong number of files
    if max(number_of_files,len(os.listdir("./"+version))) != number_of_files:
        print(f'{version} is not the same as {available_versions[0]}')
        break
    
    

'''
if python_version in available_versions:
    print(f"PyInstaller est disponible localement pour {python_version}.")
    print("Installation de PyInstaller...")
    
    if installVersion(libs_path,python_version):
        print("PyInstaller installé avec succès.")
        
    else:
        print("Erreur lors de l'installation de PyInstaller.")
    

if python_global_version in available_global_versions:
    print(f"PyInstaller n'est pas disponible localement pour {python_version}, mais il y a une version pour {python_global_version}.")
    print("Installation de PyInstaller...")
    
    if installVersion(libs_path,python_global_version):
        print("PyInstaller installé avec succès.")
        
    else:
        print("Erreur lors de l'installation de PyInstaller.")
    
else:
    print(f"PyInstaller n'est pas disponible localement pour {python_version} ou {python_global_version}.")
    print(f"Tentative de téléchargement de PyInstaller pour {python_version} depuis PyPi.org ...")
    
    if downloadVersion(libs_path,python_version):
        print(f"Téléchargement de PyInstaller pour {python_version} réussi.")
        print(f"Relancez le script pour l'installer.")
        
    else:
        print("Téléchargement impossible pour cause d'erreur réseau.")
        print("Le dossier de version a été crée, et un 'requirements.txt' y a été ajouté.")
        print("Veuillez relancer le script sur une machine connectée à internet.")
        print("Les librairies seront automatiquement téléchargées dans le dossier de version.")
        print(f"Alternativement, les librairies peuvent être placées manuellement dans le dossier {os.path.join(libs_path,python_version)}")  
'''
        

'\nif python_version in available_versions:\n    print(f"PyInstaller est disponible localement pour {python_version}.")\n    print("Installation de PyInstaller...")\n    \n    if installVersion(libs_path,python_version):\n        print("PyInstaller installé avec succès.")\n        \n    else:\n        print("Erreur lors de l\'installation de PyInstaller.")\n    \n\nif python_global_version in available_global_versions:\n    print(f"PyInstaller n\'est pas disponible localement pour {python_version}, mais il y a une version pour {python_global_version}.")\n    print("Installation de PyInstaller...")\n    \n    if installVersion(libs_path,python_global_version):\n        print("PyInstaller installé avec succès.")\n        \n    else:\n        print("Erreur lors de l\'installation de PyInstaller.")\n    \nelse:\n    print(f"PyInstaller n\'est pas disponible localement pour {python_version} ou {python_global_version}.")\n    print(f"Tentative de téléchargement de PyInstaller pour {python_ve

### Guide d'Usages

#### 1. La commande PyInstaller



```bash 
pyinstaller [options] script [script …] | specfile
```
ou sur Windows, si pyinstaller n'est pas dans le $PATH :
```bash
py -m PyInstaller [options] script [script …] | specfile
```

Cette commande permet de package les scripts python en .exe.


Dans son utilisation la plus simple, on se place dans le dossier contenant le script et on lance :

```bash 
pyinstaller myscript.py
```



PyInstaller analyse le script et :
- Génère `myscript.spec` dans le même dossier que le script.
- Crée un dossier `build` dans le même dossier que le script s'il n'existe pas.
- Génère des fichiers de log et de travail dans le dossier `build`.
- Crée un dossier dist dans le même dossier que le script s'il n'existe pas.
- Génère l'exécutable `myscript` dans le dossier `dist`. C'est ce .exe que l'on peut ensuite distribuer à l'utilisateur.

**TLDR : PyInstaller analyse le script et génère un dossier `dist` contenant l'exécutable.**


#### 2. Cas d'usages


- Cas 1 : On génère un bundle avec un nom particulier à partir d'un script dans le current directory <br>
        ```bash	pyinstaller -n myscript_Winx64_python3.10.11.exe myscript.py```

- Cas 2 : Le script est dans le current directory, on génère un fichier .exe avec un nom particulier dans un directory spécifique <br>
        ```bash	pyinstaller --distpath "./exe" -F -n myscript_Winx64_python3.10.11.exe myscript.py```

- Cas 3 : Le script est dans un dossier spécifique, on génère un bundle dans le current directory et le .spec dans un dossier spécifique<br>
        ```bash	pyinstaller --specpath=<chemin_du_dossier_spec> --distpath=. --workpath=<chemin_du_dossier_de_travail> <chemin_du_script>```

#### 3. Quelques options utiles


- ```bash -h, --help ``` ->
Affiche l'aide et quitte.

- ```bash -v, --version``` ->
Affiche les infos de version et quitte.

- ```bash -D, --onedir``` ->
Crée un dossier contenant l'exécutable (défaut)

- ```bash -F, --onefile``` ->
Crée un fichier exécutable unique.

- ```bash --specpath DIR``` ->
Dossier où stocker le fichier spec généré (défaut : dossier courant)

- ```bash -n NAME, --name NAME``` ->
Nom à assigner à l'application packagée et au fichier spec (défaut : nom du premier script)

- ```bash --distpath DIR``` ->
Où mettre l'application packagée (défaut : ./dist)

- ```bash --workpath WORKPATH``` ->
Où mettre tous les fichiers temporaires de travail, .log, .pyz et etc. (défaut : ./build)

- ```bash -y, --noconfirm``` ->
Remplace le dossier de sortie (défaut : SPECPATH/dist/SPECNAME) sans demander de confirmation

- ```bash -a, --ascii``` ->
Ne pas inclure le support de l'encodage unicode dans l'.exe (défaut : inclus si disponible)

- ```bash --clean``` ->
Nettoie le cache PyInstaller et supprime les fichiers temporaires avant de construire.